In [3]:
import sys
sys.path.append('../..')

In [4]:
import pandas as pd

In [6]:
def appendPrefix(attribute,prefix):
    return list(map(lambda s:prefix+s,attribute.split()))

In [7]:
def preProcessNeighbors(nn,opposite_label_data,attribute):
    nn_values = pd.merge(nn,opposite_label_data,left_on=attribute,right_on='id')
    left_values, right_values  = nn_values['ltable_'+attribute],nn_values['rtable_'+attribute]
    left_values_prefixed = list(map(lambda att:tuple(appendPrefix(att,'L_')),left_values))
    right_values_prefixed = list(map(lambda att:tuple(appendPrefix(att,'R_')),right_values))
    return left_values_prefixed,right_values_prefixed

In [15]:
pos_nn = pd.read_csv('experiment-results/positives_nn_onlytrain_hybrid.csv')
train_neg = pd.read_csv('../../Structured/Beer/train_negatives.csv')
lval,rval = preProcessNeighbors(pos_nn,train_neg,'Beer_Name')
transactions = list(map(lambda l,r:l+r,lval,rval))

In [23]:
from efficient_apriori import apriori
itemsets, rules = apriori(transactions, min_support=0.2,  min_confidence=0.8)

# Print out every rule with 2 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs)==2, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

{L_Red, R_Red} -> {L_Ale, R_Ale} (conf: 0.919, supp: 0.500, lift: 1.041, conv: 1.451)
{L_Amber, R_Amber} -> {L_Ale, R_Ale} (conf: 1.000, supp: 0.397, lift: 1.133, conv: 117647058.824)
{L_Red, R_Imperial} -> {L_Ale, R_Ale} (conf: 1.000, supp: 0.338, lift: 1.133, conv: 117647058.824)
{R_Imperial, R_Red} -> {L_Ale, R_Ale} (conf: 1.000, supp: 0.338, lift: 1.133, conv: 117647058.824)
{L_Imperial, L_Red} -> {L_Ale, R_Ale} (conf: 1.000, supp: 0.324, lift: 1.133, conv: 117647058.824)
{L_Imperial, R_Imperial} -> {L_Ale, R_Ale} (conf: 1.000, supp: 0.324, lift: 1.133, conv: 117647058.824)
{L_Imperial, R_Red} -> {L_Ale, R_Ale} (conf: 1.000, supp: 0.324, lift: 1.133, conv: 117647058.824)
{L_Ale, L_Imperial} -> {L_Red, R_Red} (conf: 1.000, supp: 0.324, lift: 1.838, conv: 455882352.941)
{L_Ale, R_Imperial} -> {L_Red, R_Red} (conf: 1.000, supp: 0.338, lift: 1.838, conv: 455882352.941)
{L_Imperial, R_Ale} -> {L_Red, R_Red} (conf: 1.000, supp: 0.324, lift: 1.838, conv: 455882352.941)
{L_Imperial, R_Impe